##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Uso do TensorBoard em Notebooks

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tensorboard/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

O TensorBoard pode ser usado diretamente em experiências de notebooks, como [Colab](https://colab.research.google.com/) e [Jupyter](https://jupyter.org/). Isso pode ser útil para compartilhar resultados, integrando o TensorBoard a fluxos de trabalho existentes e usando o TensorBoard sem instalar nada localmente.

## Configuração

Comece instalando o TF 2.0 e carregando a extensão de notebook do TensorBoard:

**Para usuários do Jupyter:** se você instalou o Jupyter e o TensorBoard no mesmo virtualenv, deve estar tudo pronto. Se estiver usando uma configuração mais complicada, como uma instalação global do Jupyter e kernels para diferentes ambientes Conda/virtualenv, é preciso garantir que o binário do `tensorboard` esteja no seu `PATH` dentro do contexto do notebook do Jupyter. Uma maneira de fazer isso é modificar o `kernel_spec` para inserir o diretório `bin` do ambiente no início do `PATH`, [conforme descrito aqui](https://github.com/ipython/ipykernel/issues/395#issuecomment-479787997).


**Para usuários do Docker:** caso você esteja executando uma imagem [Docker](https://docs.docker.com/install/) do [servidor do notebook do Jupyter usando a versão noturna do TensorFlow](https://www.tensorflow.org/install/docker#examples_using_cpu-only_images), será necessário expor a porta do notebook e do TensorBoard. Por isso, execute o container com o seguinte comando:

```
docker run -it -p 8888:8888 -p 6006:6006 \
tensorflow/tensorflow:nightly-py3-jupyter
```

quando `-p 6006` é a porta padrão do TensorBoard. Isso alocará uma porta para você executar a instância do TensorBoard. Para ter instâncias simultâneas, é necessário alocar mais portas. Além disso, passe `--bind_all` a `%tensorboard` para expor a porta fora do container.

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

Importe o TensorFlow, datetime e so:

In [ ]:
import tensorflow as tf
import datetime, os

## TensorBoard em notebooks

Baixe e dimensione o dataset [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist):

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4423680/4422102 [==============================] - 0s 0us/step


Crie um modelo bastante simples:

In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

Treine o modelo usando Keras e a callback do TensorBoard:

In [6]:
def train_model():
  
  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train, 
            y=y_train, 
            epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

train_model()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 182us/sample - loss: 0.4976 - accuracy: 0.8204 - val_loss: 0.4143 - val_accuracy: 0.8538
Epoch 2/5
60000/60000 [==============================] - 10s 174us/sample - loss: 0.3845 - accuracy: 0.8588 - val_loss: 0.3855 - val_accuracy: 0.8626
Epoch 3/5
60000/60000 [==============================] - 10s 175us/sample - loss: 0.3513 - accuracy: 0.8705 - val_loss: 0.3740 - val_accuracy: 0.8607
Epoch 4/5
60000/60000 [==============================] - 11s 177us/sample - loss: 0.3287 - accuracy: 0.8793 - val_loss: 0.3596 - val_accuracy: 0.8719
Epoch 5/5
60000/60000 [==============================] - 11s 178us/sample - loss: 0.3153 - accuracy: 0.8825 - val_loss: 0.3360 - val_accuracy: 0.8782


Inicie o TensorBoard no notebook usando [magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html):

In [ ]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard.png?raw=1"/> -->

Agora você pode ver painéis de controle, como **Time Series**, **Graphs** (Grafos), **Distributions** (Distribuições) e outros. Alguns ainda não estão disponíveis no Colab (como o plugin de perfil).

O magic `%tensorboard` tem exatamente o mesmo formato que a invocação de linha de comando do TensorBoard, mas com um sinal `%` antes dele.

Você também pode inicializar o TensorBoard antes do treinamento para monitorá-lo em andamento:

In [ ]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_two_runs.png?raw=1"/> -->

O mesmo back-end do TensorBoard é reutilizado ao dar o mesmo comando. Se fosse escolhido um diretório de logs diferente, seria aberta uma nova instância do TensorBoard. As portas são gerenciadas automaticamente.

Comece treinando um novo modelo e veja o TensorBoard atualizar automaticamente a cada 30 segundos ou atualize com o botão no canto superior direito:

In [9]:
train_model()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 184us/sample - loss: 0.4968 - accuracy: 0.8223 - val_loss: 0.4216 - val_accuracy: 0.8481
Epoch 2/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3847 - accuracy: 0.8587 - val_loss: 0.4056 - val_accuracy: 0.8545
Epoch 3/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3495 - accuracy: 0.8727 - val_loss: 0.3600 - val_accuracy: 0.8700
Epoch 4/5
60000/60000 [==============================] - 11s 179us/sample - loss: 0.3282 - accuracy: 0.8795 - val_loss: 0.3636 - val_accuracy: 0.8694
Epoch 5/5
60000/60000 [==============================] - 11s 176us/sample - loss: 0.3115 - accuracy: 0.8839 - val_loss: 0.3438 - val_accuracy: 0.8764


Você pode usar as APIs `tensorboard.notebook` para um pouco mais de controle:

In [10]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs (started 0:00:54 ago; pid 265)


In [ ]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000) 

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_tall.png?raw=1"/> -->